In [ ]:
import sys
from pathlib import Path
import pandas as pd

# Add project root to Python path
project_root = Path().resolve().parent
sys.path.insert(0, str(project_root))

In [ ]:
import src.load as load

sensor_locations = load.load_sensor_locations()
sensor_locations.info

In [ ]:
import src.clean as clean

clean.clean_sensor_locations(sensor_locations)
sensor_locations.dtypes

print(
    sensor_locations[['location_id', 'sensor_name']]
    .groupby(['sensor_name'])
    .count()
)